In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

import tensorflow as tf
from tensorflow.keras import models, layers, utils, losses

In [2]:
# colab on google drive 

from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/groom/semi-3/fifa_pay_prediction/Data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/groom/semi-3/fifa_pay_prediction/Data


In [3]:
train = pd.read_csv("FIFA_train.csv")
test = pd.read_csv("FIFA_test.csv")

submission = pd.read_csv("submission.csv")

In [4]:
# preprocessing drop feature

del train['id']
del train['name']
del train['contract_until']

del test['id']
del test['name']
del test['contract_until']


In [5]:
target_data = train['value']

#del 'value'
del train['value']

train_data = train

In [6]:
train_data

,age,continent,position,prefer_foot,reputation,stat_overall,stat_potential,stat_skill_moves
0,31,south america,ST,left,5.0,94,94,4.0
1,27,europe,GK,right,4.0,91,93,1.0
2,31,south america,ST,right,5.0,91,91,3.0
3,32,europe,DF,right,4.0,91,91,3.0
4,25,europe,GK,right,3.0,90,93,1.0
...,...,...,...,...,...,...,...,...
8927,18,africa,MF,right,1.0,48,63,3.0
8928,19,europe,DF,right,1.0,47,59,2.0
8929,18,south america,DF,right,1.0,47,64,2.0
8930,18,europe,GK,right,1.0,47,65,1.0


In [7]:
print(train_data['continent'].unique())
print(train_data['position'].unique())
print(train_data['prefer_foot'].unique())

['south america' 'europe' 'africa' 'asia' 'oceania']
['ST' 'GK' 'DF' 'MF']
['left' 'right']


In [8]:
con_n = list(train_data['continent'].unique())
pos_n = list(train_data['position'].unique())
pref_n = list(train_data['prefer_foot'].unique())

In [9]:
length_data = len(train_data)

# 벡터해석을 위한 숫자 정보로 바꾸기
for i in range(length_data):
  if train_data['prefer_foot'][i] == 'left':
    train_data['prefer_foot'][i] = 0

  elif train_data['prefer_foot'][i] == 'right':
    train_data['prefer_foot'][i] = 1


for i in range(length_data):
  if train_data['position'][i] == 'ST':
    train_data['position'][i] = 0

  elif train_data['position'][i] == 'GK':
    train_data['position'][i] = 1

  elif train_data['position'][i] == 'DF':
    train_data['position'][i] = 2

  elif train_data['position'][i] == 'MF':
    train_data['position'][i] = 3

for i in range(length_data):
  if train_data['continent'][i] == 'south america':
    train_data['continent'][i] = 0

  elif train_data['continent'][i] == 'europe':
    train_data['continent'][i] = 1

  elif train_data['continent'][i] == 'africa':
    train_data['continent'][i] = 2

  elif train_data['continent'][i] == 'asia':
    train_data['continent'][i] = 3

  elif train_data['continent'][i] == 'oceania':
    train_data['continent'][i] = 4

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCop

In [10]:
train_data
target_data

0       110500000.0
1        72000000.0
2        80000000.0
3        51000000.0
4        68000000.0
           ...     
8927        60000.0
8928        40000.0
8929        50000.0
8930        50000.0
8931        60000.0
Name: value, Length: 8932, dtype: float64

In [11]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8932 entries, 0 to 8931
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               8932 non-null   int64  
 1   continent         8932 non-null   object 
 2   position          8932 non-null   object 
 3   prefer_foot       8932 non-null   object 
 4   reputation        8932 non-null   float64
 5   stat_overall      8932 non-null   int64  
 6   stat_potential    8932 non-null   int64  
 7   stat_skill_moves  8932 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 558.4+ KB


In [12]:
# 데이터 타입 바꾸기 
train_data['continent'] = train_data['continent'].astype(int)
train_data['position'] = train_data['position'].astype(int)
train_data['prefer_foot'] = train_data['prefer_foot'].astype(int)

# target_data = target_data.astype(np.float32)
# train_data = train_data.astype(np.float32)

In [13]:
# on hot vector

def on_hot_vector(series):
  num = np.unique(series, axis=0)
  num = num.shape[0]

  encoding = np.eye(num)[series]
  en_df = pd.DataFrame(encoding)

  make_name =[]
  for i in range(num):
    
    make_name.append(series.name + str(i))

  en_df.columns = make_name
  return en_df

In [14]:
#concat
continent_f = on_hot_vector(train_data['continent'])
position_f = on_hot_vector(train_data['position'])
prefer_foot = on_hot_vector(train_data['prefer_foot'])

vector_df = pd.concat([position_f,continent_f,prefer_foot],axis=1)

In [15]:
del train_data['continent']
del train_data['position']
del train_data['prefer_foot']

In [16]:
train_data = pd.concat([train_data,vector_df],axis = 1)

In [17]:
len(train_data.iloc[0])

16

In [18]:
#Sequential
model = models.Sequential()

# Regression
model.add(layers.Dense(input_dim=16, units=32, activation='relu'))
model.add(layers.Dense(units=32, activation='relu'))
model.add(layers.Dense(units=1, activation=None))

model.compile(optimizer='adam',
              loss=losses.mean_squared_error,
              metrics=['mean_squared_error']) 

# summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                544       
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,633
Trainable params: 1,633
Non-trainable params: 0
_________________________________________________________________


In [19]:
# model fitting 
model.fit(train_data,
          target_data,
          batch_size = 100, #default batch_size = 32
          epochs = 50)

Epoch 1/50
90/90 [==============================] - 1s 2ms/step - loss: 41833514139648.0000 - mean_squared_error: 41833514139648.0000
Epoch 2/50
90/90 [==============================] - 0s 2ms/step - loss: 41829118509056.0000 - mean_squared_error: 41829118509056.0000
Epoch 3/50
90/90 [==============================] - 0s 2ms/step - loss: 41813700247552.0000 - mean_squared_error: 41813700247552.0000
Epoch 4/50
90/90 [==============================] - 0s 2ms/step - loss: 41778262573056.0000 - mean_squared_error: 41778262573056.0000
Epoch 5/50
90/90 [==============================] - 0s 2ms/step - loss: 41714479792128.0000 - mean_squared_error: 41714479792128.0000
Epoch 6/50
90/90 [==============================] - 0s 2ms/step - loss: 41615909453824.0000 - mean_squared_error: 41615909453824.0000
Epoch 7/50
90/90 [==============================] - 0s 2ms/step - loss: 41477765857280.0000 - mean_squared_error: 41477765857280.0000
Epoch 8/50
90/90 [==============================] - 0s 2ms/ste

In [19]:
model.predict